In [1]:
import pandas as pd
import numpy as np

In [2]:
X,y = pd.read_csv('data/X_prepped.csv', index_col = 'id'), pd.read_csv('data/Y_train.csv', index_col = 'id')

In [3]:
import random
random.seed(42)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb

import optuna
from optuna import Trial, visualization

In [5]:
!nvidia-smi

Wed May 25 21:23:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P5    21W / 250W |   1033MiB /  8192MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
def Objective(trial):

    
    # n_estimators (int) – Number of gradient boosted trees.
    # max_depth (int) – Maximum tree depth for base learners.
    # learning_rate (float) – Boosting learning rate.
    # booster (string) – Specify which booster to use: gbtree, gblinear or dart.
    # tree_method (string) – Specify which tree method to use.
    # gamma (float) – Minimum loss reduction required to make a further partition on a leaf node of the tree.
    # min_child_weight (float) – Minimum sum of instance weight(hessian) needed in a child.
    # max_delta_step (float) – Maximum delta step we allow each tree’s weight estimation to be.
    # subsample (float) – Subsample ratio of the training instance.
    # colsample_bytree (float) – Subsample ratio of columns when constructing each tree.
    # colsample_bylevel (float) – Subsample ratio of columns for each level.
    # colsample_bynode (float) – Subsample ratio of columns for each split.
    # reg_alpha (float) – L1 regularization term on weights
    # reg_lambda (float) – L2 regularization term on weights


    param = {
        "tree_method": "gpu_hist",  # use gpu
        "objective":"binary:hinge", #Hinge loss may give better accuracy and some sparsity but much less sensitivity in terms of probabilities
        "lambda": trial.suggest_loguniform("lambda", 1e-3, 10.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-3, 10.0),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample":  trial.suggest_loguniform("subsample", 0.4, 0.8),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "n_estimators": 8000,
        'n_jobs' : -1,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "random_state": 42,
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 100),
    }

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)
    clf = xgb.XGBClassifier(**param, use_label_encoder=False)
    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
    )

    preds = clf.predict(X_valid)
    auc = roc_auc_score(y_valid, preds)
    return auc

In [10]:
study = optuna.create_study(direction="maximize", study_name="xgb opt")
study.optimize(Objective, n_trials=50) 

[I 2022-05-25 21:27:52,125] A new study created in memory with name: xgb opt


[0]	validation_0-error:0.51172
[1]	validation_0-error:0.51172
[2]	validation_0-error:0.51172
[3]	validation_0-error:0.51172
[4]	validation_0-error:0.51172
[5]	validation_0-error:0.51172
[6]	validation_0-error:0.51172
[7]	validation_0-error:0.51172
[8]	validation_0-error:0.51172
[9]	validation_0-error:0.51172
[10]	validation_0-error:0.51172
[11]	validation_0-error:0.51172
[12]	validation_0-error:0.51172
[13]	validation_0-error:0.51172
[14]	validation_0-error:0.51172
[15]	validation_0-error:0.51172
[16]	validation_0-error:0.51172
[17]	validation_0-error:0.51172
[18]	validation_0-error:0.51172
[19]	validation_0-error:0.51159
[20]	validation_0-error:0.51101
[21]	validation_0-error:0.51056
[22]	validation_0-error:0.51000
[23]	validation_0-error:0.50747
[24]	validation_0-error:0.50446
[25]	validation_0-error:0.50137
[26]	validation_0-error:0.49798
[27]	validation_0-error:0.49524
[28]	validation_0-error:0.49422
[29]	validation_0-error:0.49234
[30]	validation_0-error:0.48923
[31]	validation_0-

In [ ]:
best_param = study.best_trial.params
best_param